<a href="https://colab.research.google.com/github/Hamza-Ali0237/HF-Projects/blob/main/Intermediate/Conversational%20AI%20App/Chatbot_With_Memory_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [17]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-large-seq2seq")
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-large-seq2seq")

In [18]:
# t2t_pipeline = pipeline(
#     task = "text2text-generation",
#     model = model,
#     tokenizer = tokenizer
# )

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32102, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32102, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [20]:
# store conversation history
history = []

In [21]:
def generate_response(user_prompt):

  instruction = user_prompt

  context = " ".join([f"User: {u} Assistant: {a}" for u, a in history])

  prompt = f"{instruction} [SEP] {context}"

  inputs = tokenizer(
      prompt,
      return_tensors='pt',
      max_length = 1024,
      truncation = True
  )

  inputs = {k: v.to(device) for k, v in inputs.items()}

  outputs = model.generate(
      **inputs,
      max_length = 256,
      do_sample = True,
      top_p = 0.9,
      temperature = 0.7
  )

  response = tokenizer.decode(
      outputs[0],
      skip_special_tokens = True
  )

  history.append((user_prompt, response))

  return response

In [22]:
if __name__ == "__main__":
    print("GODEL Chatbot: Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() in {"exit", "quit"}:
            break
        reply = generate_response(user_input)
        print(f"Bot: {reply}\n")

GODEL Chatbot: Type 'exit' to quit.

You: heyyy, whats up?
Bot: Hi, just got back from the gym.

You: noice man, getting them gains
Bot: I'm just getting back from the gym.

You: I know. what did you do in the gym?
Bot: I just got back from the gym, and I'm still getting those gains.

You: bro. just shut up
Bot: The 'getting them gains' part is funny because I was a bit of a troll.

You: lol
Bot: I'm not a troll, but I do find it funny that you got back from the gym.

You: exit


# Gradio UI

In [23]:
!pip install gradio

In [24]:
import gradio as gr

# Global conversation history
ui_history = []

def chatbot(user_prompt):

    context = " ".join([f"User: {u} Assistant: {a}" for u, a in ui_history])

    prompt = f"{user_prompt} [SEP] {context}"

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    )

    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs,
        max_length=256,
        do_sample=True,
        top_p=0.9,
        temperature=0.7
    )

    response = tokenizer.decode(
        outputs[0],
        skip_special_tokens=True
    )

    ui_history.append((user_prompt, response))

    messages = []
    for u, a in ui_history:
        messages.append({"role": "user", "content": u})
        messages.append({"role": "assistant", "content": a})

    return messages

def clear_history():
    ui_history.clear()
    return []

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("# 🤖 GODEL Chatbot with Memory")
    chatbot_ui = gr.Chatbot(type="messages")
    with gr.Row():
        txt = gr.Textbox(placeholder="Type a message...")
        submit_btn = gr.Button("Send")
        clear_btn = gr.Button("Clear")

    submit_btn.click(fn=chatbot, inputs=txt, outputs=chatbot_ui)
    txt.submit(fn=chatbot, inputs=txt, outputs=chatbot_ui)
    clear_btn.click(fn=clear_history, inputs=None, outputs=chatbot_ui)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://57bf5bacb7f5c430e2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


The app is working fine, but for better responses I would suggest changing the model to a better one.